# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

In [19]:
import numpy as np 
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import bs4 as bs
import requests
import xlsxwriter
import math
import pickle
from scipy import stats

In [20]:
portfolio_size = 100.0

In [21]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        # Remove stock class symbol "BRK.B"
        if "." in ticker:
            ticker = ticker.replace(".", "-")
        tickers.append(ticker[:-1])
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers


In [22]:
# Retrieve sp 500 ticker data from yf
sp_500_tickers = save_sp500_tickers()
tickers = yf.Tickers(' '.join(sp_500_tickers))

In [23]:
today = datetime.today().strftime('%Y-%m-%d')
start_of_year = datetime(datetime.today().year - 1, 
                         datetime.today().month, 
                         datetime.today().day).strftime('%Y-%m-%d')
today, start_of_year

('2021-02-15', '2020-02-15')

In [24]:
data = yf.download(sp_500_tickers, start=start_of_year, end=today)


[*********************100%***********************]  505 of 505 completed


In [25]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]    
symbol_groups = list(chunks(sp_500_tickers, 100))
symbol_strings = []
# Each cluster is turned into a CSV string
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [26]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([
                symbol, 
                data["Close"][symbol][-1],
                100* (data["Close"][symbol][-1] / data["Close"][symbol][0] - 1),
                'N/A'],
                index = my_columns),
            ignore_index = True)

In [27]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)

# HQM analysis

In [28]:
# calculate with HQM
hqm_columns = [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy', 
    'One-Year Price Return', 
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    for symbol in symbol_string.split(','):
        last_price = data["Close"][symbol][-1]
        start_price = data["Close"][symbol][0]
        six_month_price = data["Close"][symbol][125]
        three_month_price = data["Close"][symbol][188]
        one_month_price = data["Close"][symbol][229]
        one_year_percent_return = 100 * (last_price / start_price - 1)
        six_month_percent_return = 100 * (last_price / six_month_price - 1)
        three_month_percent_return = 100 * (last_price / three_month_price - 1)
        one_month_percent_return = 100 * (last_price / one_month_price - 1)
        
        hqm_dataframe = hqm_dataframe.append(
            pd.Series([symbol, 
                       last_price,
                       'N/A',
                       one_year_percent_return,
                       'N/A',
                       six_month_percent_return,
                       'N/A',
                       three_month_percent_return,
                       'N/A',
                       one_month_percent_return,
                       'N/A',
                       'N/A'
                       ], 
                      index = hqm_columns), 
            ignore_index = True)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

In [29]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f"{time_period} Return Percentile"] = stats.percentileofscore(
            hqm_dataframe[f"{time_period} Price Return"], 
            hqm_dataframe.loc[row, f"{time_period} Price Return"]) / 100


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

In [30]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [31]:
# Select 50 best by HQM score
hqm_dataframe = hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)


In [32]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = position_size / hqm_dataframe['Price'][i]


In [33]:
# Take top 50 momentum stocks
hqm_dataframe = hqm_dataframe[:50]

In [34]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
448,TWTR,71.900002,0.002754,88.912241,0.962376,89.709759,0.970297,67.951411,0.976238,52.265988,1.0,0.977228
471,VIAC,58.310001,0.003396,66.125367,0.928713,112.345242,0.990099,99.350432,0.992079,35.227279,0.99604,0.976733
144,DISCA,47.799999,0.004143,57.755777,0.893069,111.317407,0.986139,119.266059,0.99802,30.280733,0.990099,0.966832
174,ETSY,233.860001,0.000847,338.597153,0.990099,80.922181,0.964356,84.098247,0.986139,12.959475,0.89505,0.958911
46,AMAT,116.699997,0.001697,79.097591,0.946535,72.582065,0.948515,67.191965,0.974257,19.020901,0.962376,0.957921
423,SIVB,506.500000,0.000391,93.979560,0.968317,102.292511,0.982178,52.730450,0.948515,14.543521,0.930693,0.957426
360,PYPL,298.369995,0.000664,144.025520,0.986139,55.839332,0.89505,57.784233,0.962376,21.833402,0.982178,0.956436
145,DISCK,40.750000,0.004859,41.149979,0.833663,96.290942,0.978218,106.957841,0.994059,28.346457,0.986139,0.94802
501,ZBRA,477.739990,0.000414,101.867651,0.978218,67.024435,0.940594,40.190150,0.881188,15.829792,0.940594,0.935149
244,ILMN,504.760010,0.000392,68.298209,0.934653,45.229604,0.825743,65.870334,0.970297,39.436467,0.99802,0.932178
